In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import json
import pandas as pd
import time
from skimage.morphology import skeletonize
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [7]:
def get_vortices_coordinates(mask, data):
    _, contours, hierarchy= cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    vortices_coordinates = []
    for cnt in contours:
        img = np.zeros( data.shape )
        cnt_filled = cv2.fillPoly(img, pts = [cnt], color=(255,255,255))
        
        locs = np.where(cnt_filled == 255)
        
        ''' Checking if contour is not too large ''' 
        if len(locs[0]) < 900: 
            pixels = data[locs]
            M= cv2.moments(cnt)
            try:
                cx= int(M['m10']/M['m00'])
                cy= int(M['m01']/M['m00'])
                
                """ Pixel analysis """
                pix = pd.DataFrame(data=moving_average(pixels, min(int(0.27*len(pixels)), 25)))
                zero_crossings = len(np.where(np.diff(np.signbit(pix[0])))[0])
                if zero_crossings > 0:
                    if zero_crossings > 1:
                        locs = np.where(skeletonize(cnt_filled == 255) == True)
                        pixels = data[locs]
                        pix = pd.DataFrame(data=pixels)
                        zero_crossings = len(np.where(np.diff(np.signbit(pix[0])))[0])
                        if len(pix) > 16:
                            my_locs = locs
                            pix = pd.DataFrame(data=moving_average(pixels,2))
                            zero_crossings = len(np.where(np.diff(np.signbit(pix[0])))[0])
                            range_v = range(min(zero_crossings,2))
                            if len(range_v) > 1:
                                my_locs= locs
                                modified_coordinates = [list(zip(locs[0], locs[1]))[1], list(zip(locs[0], locs[1]))[-2]]
                                for (i, (new_y, new_x)) in zip(range_v, modified_coordinates):
                                    vortices_coordinates.append((new_x, new_y))
                            else:
                                for i in range_v:
                                    vortices_coordinates.append((cx+i, cy+i))
                        else:

                            vortices_coordinates.append((cx,cy))
                    else:
                        vortices_coordinates.append((cx,cy))
                        
                elif zero_crossings == 0 and pixels.min() == -0.5 and pixels.max() == 0.5:
                    vortices_coordinates.append((cx,cy))
            except ZeroDivisionError:
                print("Zero division")
                
    return vortices_coordinates

In [8]:
def convert_coordinates_to_frame(mask, frame, coordinates):
    vortices_list = []
    for x,y in coordinates:
        cx= int(x * frame.shape[0] / mask.shape[0])
        cy= int(y * frame.shape[0] / mask.shape[0])
        vortices_list.append((cx, cy))
    return vortices_list

# EXTRACTION MOMENTS OF TIME FOR WHICH THE DATA IS LABELED

In [9]:
''' EXTRACT TIME RANGE '''
mypath = 'CalculateDetectionMetrics/cropped_data_labeled/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
time_range = np.array(sorted([float(x[19:-5]) for x in list(filter(lambda s: 'json' in s, onlyfiles))]))
time_range = time_range[time_range >= 30]
time_range = np.round(time_range, 1)
time_range

array([ 30. ,  33. ,  36. ,  39. ,  42.1,  45. ,  48. ,  51. ,  54. ,
        57. ,  60. ,  63. ,  66. ,  69. ,  72. ,  75. ,  78. ,  81. ,
        84. ,  87. ,  90. ,  93. ,  96. ,  99. , 102. , 105. , 108. ,
       111. , 114. , 117. , 120. , 123. , 126. , 129. , 132. , 135. ,
       138. , 141. , 144. , 147. , 150. , 153. , 156. , 159. , 162. ,
       165. , 168. , 171. , 174. , 177. , 180. , 183. , 186. , 189. ,
       192. , 195. , 198. , 201. , 204. , 207. , 210.1, 214. , 217. ,
       220. , 223. , 226. , 229. , 232. , 235. , 238. , 241. , 244. ,
       247. , 250. , 253. , 256. , 259. , 262. , 265. , 268. , 271. ,
       274. , 277. , 280. , 283. , 286. , 289. , 292. , 295. , 298. ,
       301. , 304. , 307. , 310. , 313. , 316. , 319. , 322. , 325. ,
       328. ])

# METHOD 5 ALGORITHM

In [10]:
program_starts = time.time()

detected_vortices = {}

for t in time_range:
    data = np.genfromtxt(f'input_data/x_vel/x_phase_velocity_snapshot_time={np.round(t, 1)}0.dat', delimiter='\t')
    frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t, 1)}0.png')
    data = np.delete(data, 818, axis=1)
    data = np.delete(data, 0, axis=0)
    data = np.delete(data, 0 ,axis = 1)
    data = np.rot90(data, 1)
    data[data > 0.4] = 0.5
    data[data < (-0.4)] = -0.5

    mask_x = np.array([-1,1])
    mask_y = mask_x.T

    Ix = cv2.filter2D(data, -1, mask_x)
    Iy = cv2.filter2D(data, -1, mask_y)
    I_xy = cv2.magnitude(Ix, Iy)

    I_xy=abs(I_xy)
    I_xy[I_xy <= 0.27] = 0
    I_xy[I_xy > 0.27] = 255
    
    ''' MORPHOLOGY OPERATIONS'''
    mask = np.uint8(I_xy)
    kernel = np.ones((5,5), dtype='uint8')
    mask = cv2.dilate(mask, kernel)
    
    ''' PIXEL ANALYSIS '''
    coordinates_after_pixel_analysis = get_vortices_coordinates(mask, data)
    
    '''GET DETECTED VORTICES COORDINATES'''
    detected_vortices[np.round(t,1)]  = convert_coordinates_to_frame(mask, frame, coordinates_after_pixel_analysis)
    
now = time.time()
print("It has been {0} seconds since the loop started".format(now - program_starts))

It has been 60.94582939147949 seconds since the loop started


# TP, FP, FN METRICS CALCULATIONS

In [22]:
''' READ JSON FILES WITH LABELED BOUNDING BOXES '''
bbox_size = 3
for index, t in enumerate(time_range):
    with open(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.json', 'r') as f:
        dictData = json.load(f)
    frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.png')
    TP = []
    for i, vortex in enumerate(dictData['shapes']):
        labeled_mask = np.zeros((frame.shape[:2]), dtype='uint8')
        a = int(vortex['points'][0][0])
        b = int(vortex['points'][0][1])
        c = int(vortex['points'][1][0])
        d = int(vortex['points'][1][1])
    #         print(f'vortex {i+1}', a, b, c, d)
        cv2.rectangle(labeled_mask, (a,b), (c,d), 255, cv2.FILLED)
        for x,y in detected_vortices[np.round(t,1)]:
            if (x,y) not in TP:
                detected_vortex = np.zeros((frame.shape[:2]), dtype='uint8')
                cv2.rectangle(detected_vortex, (x-bbox_size,y-bbox_size), (x+bbox_size,y+bbox_size), 255, cv2.FILLED)
                result = np.all(cv2.bitwise_and(detected_vortex, labeled_mask) == 0)
                if not result:
                    TP.append((x,y))
                    break
    tp_count = len(TP) #CORRECT
    fp_count = len(detected_vortices[np.round(t,1)]) - len(TP) #INCORRECT
    fn_count = len(dictData['shapes']) - len(TP) #UNDETECTED
    
    ''' INSPECTION IF CALCULATION WORK PROPERLY '''
    if index % 10 == 0:
        for (x,y) in detected_vortices[np.round(t,1)]:
            if (x,y) in TP:
                color = (0,255,0)
            else:
                color = (0,0,255)
            cv2.rectangle(frame, (x-12,y-12), (x+12,y+12), color, 2)
        for parameter, value, shift in zip(['T=', 'TP=', 'FP=', 'FN='], [t, tp_count, fp_count, fn_count], [30, 60, 90, 120]):
            cv2.putText(frame, text=parameter+str(value), org=(10, shift),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.9, color=(0,0,0),
                    thickness=1, lineType=cv2.LINE_AA)
        cv2.imwrite(f'Method5-PixelAnalysis_RESULTS/TP_FP_{np.round(t,1)}.png', frame)
        print(t)
        
    ''' SAVE RESULTS TO FILE '''
#     print(t, tp_count, fp_count, fn_count)
    with open('Method5-PixelAnalysis_RESULTS/Method5-Detection-Metrics-Results.csv', 'a') as fr:
        fr.write(f'{t}\t{tp_count}\t{fp_count}\t{fn_count}\n')

30.0
60.0
90.0
120.0
150.0
180.0
210.1
241.0
271.0
301.0


# FINAL METRICS SUMMATION

In [24]:
df_metrics = pd.read_csv('Method5-PixelAnalysis_RESULTS/Method5-Detection-Metrics-Results.csv', header=None, delimiter='\t')
df_metrics.columns = ['TIME', 'TP', 'FP', 'FN']
df_metrics.set_index('TIME', inplace=True)
df_metrics.sum()

TP    2257
FP     115
FN      96
dtype: int64

# SAVE TO FILE DETECTED COORDINATES

In [11]:
with open('BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/detected_vortices.txt','w') as data: 
      data.write(str(detected_vortices))